
## What is a Quantum Walk?

A quantum walk is the quantum counterpart of a classical random walk, where a "walker" (quantum particle) explores a network by probabilistically transitioning between nodes. Unlike classical random walks that are restricted to definite states, quantum walks exploit the principles of superposition and interference, allowing the walker to exist in multiple states simultaneously. This unique property enables quantum walks to outperform classical random walks in certain tasks, such as search algorithms and solving specific mathematical problems. Quantum walks hold immense promise for the development of powerful quantum algorithms, potentially offering significant speedups for problems considered intractable for classical computers.

## Introduction

This document delves into the details of a quantum walk on a line with 16 nodes. We will explore the theoretical underpinnings, followed by a step-by-step implementation using the Classiq framework. The key concepts covered include initializing the quantum state, defining the probability distributions for the walk, and performing the walk using quantum operators, while accounting for the specific boundary conditions of a linear topology.

## Quantum Walks: Theory

A quantum walk is the quantum analog of a classical random walk, where the walker moves between adjacent nodes (or states) with certain probabilities. However, quantum walks leverage superposition and interference, leading to fundamentally different dynamics compared to their classical counterparts.

There are two main types of quantum walks: discrete-time and continuous-time. Here, we will focus on discrete-time quantum walks.

### Discrete-Time Quantum Walk

A discrete-time quantum walk typically involves two primary operators:

**1. Coin Operator (C):** This operator acts like a quantum coin, determining the direction of the walk. A common choice for the coin operator is the Hadamard operator (H):

$$
H = \frac{1}{\sqrt{2}} \begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix}
$$

The Hadamard operator creates an equal superposition of the "heads" and "tails" states of the coin, which translates to equal probabilities for moving left or right along the line.

**2. Shift Operator (S):** This operator governs the actual movement of the walker to an adjacent node based on the coin state. For a general graph with nodes labeled from 0 to N-1, the shift operator can be defined as:

$$
S = \sum_{x=0}^{N-1} \left( |(x+1) \mod N \rangle \langle x| \otimes |0\rangle \langle 0| + |(x-1) \mod N \rangle \langle x| \otimes |1\rangle \langle 1| \right)
$$

This operator considers the specific topology of the graph (e.g., a circle or a line) to determine the possible movements.

#### Quantum Walk Operator (U)

The overall quantum walk operator (U) combines the coin and shift operators:

$$
U = S \cdot (I \otimes C)
$$

where I represents the identity operator on the position space and otimes denotes the tensor product.

#### Applying to a Line with 16 Nodes

For a line with 16 nodes, the shift operator needs to account for the boundary conditions, where nodes at the ends have fewer neighbors. The adjusted shift operator for a line with 16 nodes (0 ≤ x ≤ 15) can be defined as:

$$
S = \sum_{x=0}^{15} \left(
\begin{cases}
|x+1 \rangle \langle x| \otimes |0\rangle \langle 0| + |x-1 \rangle \langle x| \otimes |1\rangle \langle 1| & \text{if } 0 < x < 15 \\
|x+1 \rangle \langle x| \otimes |0\rangle \langle 0| & \text{if } x = 0 \\
|x-1 \rangle \langle x| \otimes |1\rangle \langle 1| & \text{if } x = 15
\end{cases}
\right)
$$

This ensures that the walker doesn't attempt to move beyond the boundaries of the line.


## Implementation

Let's begin by importing the necessary Python libraries and setting the number of qubits required to represent the 16 nodes ($log_{_2}(16) = 4$):


In [1]:
# pip install -U classiq
# import classiq
# classiq.authenticate()

In [2]:
from classiq import *
from classiq.qmod.symbolic import logical_or
from classiq.execution import ExecutionPreferences


size = 4  # Number of qubits to represent the vertices (log2 of 16 nodes)
num_nodes = 2**size  # Total number of nodes (16 nodes)




### Coin Operator

The coin operator is implemented as a series of functions:

* **`prepare_minus`:** Prepares a qubit in the minus state, which is a superposition of |0> and |1> with equal amplitudes but opposite phases.
* **`diffuser_oracle`:** Implements the diffuser oracle used in amplitude amplification.
* **`zero_diffuser`:** Combines the `prepare_minus` and `diffuser_oracle` functions.
* **`C_iteration`:** Defines the coin operation for a specific node, setting the probabilities for moving to adjacent nodes based on the node's position.
* **`C_operator`:** Applies the `C_iteration` function to all nodes, creating the complete coin operator.







In [3]:
@qfunc
def prepare_minus(x: QBit):
    """
    Prepares a qubit in the |-> state.

    Args:
        x (QBit): The qubit to prepare.
    """
    X(x)
    H(x)

@qfunc
def diffuser_oracle(aux: Output[QNum], x: QNum):
    """
    Implements a diffuser oracle for Grover's algorithm.

    Args:
        aux (Output[QNum]): Auxiliary qubit.
        x (QNum): QNum representing the node to check.
    """
    aux ^= (x != 0)

@qfunc
def zero_diffuser(x: QNum):
    """
    Implements the zero diffuser for the quantum walk.

    Args:
        x (QNum): QNum representing the current node.
    """
    aux = QNum('aux')
    allocate(1, aux)
    within_apply(compute=lambda: prepare_minus(aux),
                action=lambda: diffuser_oracle)
    
def C_iteration(i: int, vertices: QNum, adjacent_vertices: QNum):
    """
    Defines the coin operation for the quantum walk.

    Args:
        i (int): Current node index.
        vertices (QNum): QNum representing the vertices.
        adjacent_vertices (QNum): QNum representing the adjacent vertices.
    """
    prob = [0] * num_nodes  # Initialize probability vector for 16 nodes
    if 0 < i < num_nodes - 1:
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5
    elif i == 0:
        prob[i + 1] = 1.0
    elif i == num_nodes - 1:
        prob[i - 1] = 1.0
    print(f'Node={i}, prob vec ={prob}')
    control(ctrl=vertices == i,
           operand=lambda: within_apply(
               compute=lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
               action=lambda: zero_diffuser(adjacent_vertices)))

@qfunc
def C_operator(vertices: QNum, adjacent_vertices: QNum):
    """
    Applies the coin operator to all vertices.

    Args:
        vertices (QNum): QNum representing the vertices.
        adjacent_vertices (QNum): QNum representing the adjacent vertices.
    """
    for i in range(num_nodes):
        C_iteration(i, vertices, adjacent_vertices)



### Shift Operator

The shift operator is implemented using:

* **`edge_oracle`:** Checks if two vertices are adjacent.
* **`bitwise_swap`:** Swaps the values of two QArrays bit by bit.
* **`S_operator`:** Applies the shift operation based on the edge oracle and bitwise swap.

In [4]:
@qfunc
def edge_oracle(res: Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    """
    Oracle that checks if two vertices are adjacent.

    Args:
        res (Output[QBit]): Result bit to store the adjacency check.
        vertices (QNum): QNum representing the vertices.
        adjacent_vertices (QNum): QNum representing the adjacent vertices.
    """
    res |= (((vertices - adjacent_vertices) ** 2 == 1))

@qfunc
def bitwise_swap(x: QArray[QBit], y: QArray[QBit]):
    """
    Swaps the values of two QArrays bit by bit.

    Args:
        x (QArray[QBit]): First QArray of qubits.
        y (QArray[QBit]): Second QArray of qubits.
    """
    repeat(count=x.len,
           iteration=lambda i: SWAP(x[i], y[i]))

@qfunc
def S_operator(vertices: QNum, adjacent_vertices: QNum):
    """
    Applies the shift operator to swap vertices with their adjacent vertices.

    Args:
        vertices (QNum): QNum representing the vertices.
        adjacent_vertices (QNum): QNum representing the adjacent vertices.
    """
    res = QNum('res')
    edge_oracle(res, vertices, adjacent_vertices)
    control(ctrl=res == 1,
           operand=lambda: bitwise_swap(vertices, adjacent_vertices))


## Putting It Together

### Main Quantum Walk Function

The `main` function orchestrates the quantum walk:

1. Allocates qubits for vertices and adjacent vertices.
2. Initializes vertices in a superposition using the Hadamard transform.
3. Applies the coin and shift operators.












In [5]:
@qfunc
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
    """
    Main function to perform the quantum random walk.

    Args:
        vertices (Output[QNum]): Output QNum to represent vertices.
        adjacent_vertices (Output[QNum]): Output QNum to represent adjacent vertices.
    """
    allocate(size, vertices)  # Allocate qubits for vertices
    hadamard_transform(vertices)  # Apply Hadamard transform to initialize superposition
    allocate(size, adjacent_vertices)  # Allocate qubits for adjacent vertices

    C_operator(vertices, adjacent_vertices)  # Apply coin operator
    S_operator(vertices, adjacent_vertices)  # Apply shift operator

### Creating the Quantum Model

We create the quantum model and synthesize the quantum program:

In [6]:
# Create and synthesize the quantum model
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Node=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
Node=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
Node=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
Node=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
Node=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
Node=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
Node=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
Node=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0]
Node

Opening in existing browser session.
